<a href="https://colab.research.google.com/github/ahsank/StockML/blob/main/CUDATest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [30]:
!nvidia-smi

Wed May  1 18:11:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmprtc7g9od".


In [29]:
%%cuda
// https://stackoverflow.com/questions/7663343/simplest-possible-example-to-show-gpu-outperform-cpu-using-cuda
#include <iostream>
#include <chrono>

#define N (256*256)
#define M (10000)

float cpufn() {
    float data[N]; int count = 0;
    for(int i = 0; i < N; i++)
    {
      data[i] = 1.0f * i / N;
      for(int j = 0; j < M; j++)
      {
          data[i] = data[i] * data[i] - 0.25f;
      }
    }
    return data[0];
}

__global__ void cudakernel(float *buf)
{
     int i = threadIdx.x + blockIdx.x * blockDim.x;
     buf[i] = 1.0f * i / N;
     for(int j = 0; j < M; j++)
        buf[i] = buf[i] * buf[i] - 0.25f;
}

float gpufn() {
    float data[N]; int count = 0;
    float *d_data;
    cudaMalloc(&d_data, N * sizeof(float));
    cudakernel<<<N/256, 256>>>(d_data);
    cudaMemcpy(data, d_data, N * sizeof(float), cudaMemcpyDeviceToHost);
    cudaFree(d_data);
    return data[0];
}

template <class Fn>
void measure(std::string context, Fn fn) {
  auto begin = std::chrono::steady_clock::now();
  fn();
  std::cout << "Time difference (" << context << ")= "
    << std::chrono::duration_cast<std::chrono::milliseconds>(std::chrono::steady_clock::now() - begin).count() << "[ms]" << std::endl;
}

int main()
{
    measure("cpu", cpufn);
    measure("gpu", gpufn);
    return 0;
}

Time difference (cpu)= 3185[ms]
Time difference (gpu)= 122[ms]

